In [1]:

# Notebook must be in root in order to import app.db
from app.db import SessionLocal, Member
from app.visualize import plot_exit_pie, plot_moving_avg

import plotly.express as px

In [2]:
session = SessionLocal()

In [3]:
plot_exit_pie(session, 365).show()

In [4]:
plot_moving_avg(session, 90, 180)